In [64]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, JSONAttribute, BinaryAttribute
import json, pickle, logging, hashlib, os, zlib, lzma
from datetime import datetime
class Cacher(Model):
  """
  cache for function
  """
  cacheKey = UnicodeAttribute(hash_key=True)
  data = JSONAttribute(default={})
  compressedData = BinaryAttribute(null=True)
  timestamp = NumberAttribute()
  def __repr__(self):
    return json.dumps({
        'cacheKey': self.cacheKey,
        'data': self.data,
        'timestamp': self.timestamp
                })

  @staticmethod
  def hashValue(inputDict:dict):
      return hashlib.sha256(json.dumps(inputDict).encode()).hexdigest()

  @classmethod
  def getCache(cls, input, timeout = 86400, verbose=False, compression = True):
    # check cache for value
    cache = next(cls.query(cls.hashValue(input)), None)
    if cache and (datetime.now().timestamp() - cache.timestamp < timeout):
      logging.debug('log found')
      if not compression:
        return cache.data
      try:
        return cls.decompress(cache.compressedData)
      except:
        logging.exception('error decompressiong, perhaps data is not compressed?')
        return cache.data
    else:
      logging.warning('cache not found or expired')
      return None
  @classmethod
  def addCache(cls, input:dict, output:dict, compression = True):
    cache = cls(
        cacheKey = cls.hashValue(input),
        data = output if not compression else {},
        timestamp = datetime.now().timestamp(),
        compressedData = cls.compress(input) if compression else None
    )
    try:
        return cache.save()
    except Exception as e:
        logging.exception(f'{e}')
  @staticmethod
  def compress(inputDict:dict,method = zlib)->bin:
    return zlib.compress(json.dumps(inputDict).encode())
  @staticmethod
  def decompress(data:bin, method = zlib)->dict:
    return json.loads(zlib.decompress(data).decode())



In [65]:
# %%timeit
def testmethod(method):
  %timeit compressed = Cacher.compress([i for i in range(100000)],method = method)
  compressed = Cacher.compress([i for i in range(100000)], method = method)
  %timeit decompressed = Cacher.decompress(compressed, method = method)
  decompressed = Cacher.decompress(compressed, method = method)
# decompressed

In [66]:
# testmethod(lz4.frame)
# testmethod(zlib)
# testmethod(gzip)
# testmethod(lzma)

36.6 ms ± 398 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
8.99 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
35.6 ms ± 441 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.05 ms ± 86.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
36.3 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
8.9 ms ± 98.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
35.9 ms ± 463 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9 ms ± 91.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Test

In [2]:
#hide
import pickle
credLocation = '/Users/nic/.dynamoCache'
user =''
pw = ''
if user and pw:
  with open (credLocation , 'wb') as f:
    pickle.dump({
        'user': user,
        'pw': pw
    }, f)
with open(credLocation , 'rb') as f:
  creden = pickle.load(f)
  PW = creden['pw']
  USER = creden['user']
# print(PW, USER)

In [68]:
class Cache(Cacher):
  class Meta:
      table_name = 'dynamoCache'
      region = 'us-east-1'
      aws_access_key_id = USER
      aws_secret_access_key = PW
      billing_mode= 'PAY_PER_REQUEST'

In [75]:
Cache.addCache({'hello':'world'}, {'output': 'dummy'}, compression = False)

{'ConsumedCapacity': {'CapacityUnits': 1.0, 'TableName': 'dynamoCache'}}

In [76]:
Cache.getCache({'hello':'world'}, compression = True)

ERROR:root:error decompressiong, perhaps data is not compressed?
Traceback (most recent call last):
  File "<ipython-input-64-159a6fcca044>", line 33, in getCache
    return cls.decompress(cache.compressedData)
  File "<ipython-input-64-159a6fcca044>", line 57, in decompress
    return json.loads(zlib.decompress(data).decode())
TypeError: a bytes-like object is required, not 'NoneType'


{'output': 'dummy'}